# ランダムフォレストモデル

このノートブックでは、前処理済みデータを用いてランダムフォレストによるモデルを構築し、評価します。

- 目的変数: `price_actual`
- モデル: ランダムフォレスト（RandomForestRegressor）
- 評価指標: RMSE
- ハイパーパラメータチューニング: GridSearchCV


## 1. ライブラリのインポートとデータ読み込み

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV

# データディレクトリ
PROJECT_ROOT = Path.cwd().parent
DATA_DIR = PROJECT_ROOT / 'data'

# 前処理済みデータの読み込み
train = pd.read_csv(DATA_DIR / 'train_processed.csv')
test = pd.read_csv(DATA_DIR / 'test_processed.csv')

print('train shape:', train.shape)
print('test shape:', test.shape)

## 2. 特徴量・目的変数の設定

In [ ]:
# 目的変数
target_col = 'price_actual'

# 説明変数（目的変数とtime列以外）
drop_cols = ['time', target_col] if target_col in train.columns else ['time']
feature_cols = [col for col in train.columns if col not in drop_cols]

X = train[feature_cols]
y = train[target_col] if target_col in train.columns else train.iloc[:, -1]  # 念のため

print('Features:', feature_cols)
print('Target:', target_col)
print('X shape:', X.shape)
print('y shape:', y.shape)

## 3. 学習・検証データ分割

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
print('X_train:', X_train.shape, 'X_valid:', X_valid.shape)

## 4. ランダムフォレストモデルの学習とハイパーパラメータチューニング

In [ ]:
# ランダムフォレストモデルのインスタンス化
rf = RandomForestRegressor(random_state=42)

# ハイパーパラメータの設定
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

# GridSearchCVで最適なパラメータを探索
grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train, y_train)

print('Best parameters:', grid_search.best_params_)
print('Best score:', -grid_search.best_score_)

## 5. 最適なモデルでの予測と評価

In [ ]:
# 最適なモデルで予測
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_valid)

# RMSEで評価
rmse = mean_squared_error(y_valid, y_pred, squared=False)
print('Validation RMSE:', rmse)

## 6. テストデータへの予測と保存

In [ ]:
# テストデータの予測
X_test = test[feature_cols]
test_pred = best_model.predict(X_test)

# 予測結果の保存
submission = test[['time']].copy()
submission['price_actual_pred'] = test_pred
submission.to_csv(DATA_DIR / 'submission_random_forest.csv', index=False)
print('Saved: submission_random_forest.csv')